In [93]:
%cd /Users/maayanlab/Documents/projects/human-melanoma-signature
import sys
sys.path.append('pipeline/scripts/')
import pandas as pd
import h5py
import numpy as np
import functools, operator, math
from geode_jupies import *

/Users/maayanlab/Documents/projects/human-melanoma-signature


In [92]:
%pwd

'/Users/maayanlab/Documents/projects/human-melanoma-signature'

In [87]:
from Melanoma import *

# Code Development for Melanoma Pipeline
## Define input and output files

In [120]:
#infiles
cpm = 's1-expression_data.dir/melanoma-cpm.txt'
matches = 's2-expression_data.dir/melanoma-idmatches.txt'

In [85]:
chdir

<function geode_jupies.chdir(data, sampleclass, genes, gamma=1.0, sort=True, calculate_sig=False, nnull=10, sig_only=False, norm_vector=True)>

In [ ]:
# # Infile & outfile
# infiles = ['s1-expression_data.dir/melanoma-cpm.txt']
# outfile = 's1-expression_data.dir/melanoma-IDstatus.txt'

In [11]:
# # Infile & outfile
# infiles = ['rawdata/human_matrix.h5', 'rawdata/m2.csv']
# outfile = 's1-expression_data.dir/melanoma-cpm.txt'

### Test Code

In [45]:
# Define infiles
archs_matrix, metadata_file = infiles
# Read infile metadata_file into pandas
metadata_df = pd.read_csv(metadata_file)
# Extract all gsms from the 'id' column
allgsm = metadata_df['id'].values
# Save only the unique values by converting the array to a set first, then to a list
gsm_unique = list(set(allgsm))
# Read expression data
expression_dataframe = load_read_counts(archs_matrix, gsms = gsm_unique)
# Set index name
expression_dataframe.index.name = 'gene'
# Perform counts per million normalization
cpm_table = compute_CPMs(expression_dataframe)

In [132]:
# Read the previously generated infiles
melanoma_masterdf = pd.read_csv(melanoma_masterfile,index_col='study_index')
cpm_table = pd.read_table(cpm, index_col='gene')

# create a dictionary called matches
matches = {} 
for std_i, row in melanoma_masterdf.iterrows(): # std_i is the key of the dictionary, index of the dataframe
    ctrl_gsms = row['ctrl_id'].split(' ')
    pert_gsms = row['perturb_id'].split(' ')
    match = np.zeros(cpm_table.shape[1], dtype=np.int32) # Return a new array of given shape and type, filled with zeros.
    match[np.in1d(cpm_table.columns, ctrl_gsms)] = 1
    match[np.in1d(cpm_table.columns, pert_gsms)] = 2
    matches[std_i] = match

matches_df=pd.DataFrame(matches,index=cpm_table.columns)

In [133]:
matches

{'1': array([0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 '2_a': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 '2_b': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [126]:
matches_df=pd.read_table('s2-signatures.dir/melanoma-idmatches.txt', index_col='sample')
matches_df.head()

,1,2_a,2_b,2_c,3,5_a,5_b,5_c,11,15_a,...,17_a,17_b,17_c,18_a,18_b,18_c,19_a,19_b,19_c,19_d
sample,,,,,,,,,,,,,,,,,,,,,
GSM1847225,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GSM1846906,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GSM1846907,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GSM1484332,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GSM1484329,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
matches

{'1': array([0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 '2_a': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 '2_b': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [145]:
for sig_id, match in matches_df.items():
    print(match)

GSM1847225    0
GSM1846906    0
GSM1846907    0
GSM1484332    2
GSM1484329    1
GSM1484333    2
GSM1847223    0
GSM1846905    0
GSM1846909    0
GSM1846910    0
GSM1846915    0
GSM1846919    0
GSM1846914    0
GSM1847221    0
GSM1484330    1
GSM1846911    0
GSM1847222    0
GSM1846912    0
GSM1847224    0
GSM1846913    0
GSM1484331    1
GSM1846920    0
GSM1846916    0
GSM1484334    2
GSM1846908    0
GSM1847226    0
GSM1846918    0
GSM1846917    0
GSM1846904    0
GSM1517762    0
             ..
GSM2199887    0
GSM2199888    0
GSM2199889    0
GSM2199890    0
GSM2199891    0
GSM2199892    0
GSM2199893    0
GSM2199894    0
GSM2199895    0
GSM2199896    0
GSM2199897    0
GSM2199898    0
GSM2199899    0
GSM2199900    0
GSM2199901    0
GSM2497811    0
GSM2497812    0
GSM2497813    0
GSM2497814    0
GSM2497815    0
GSM2497816    0
GSM2497817    0
GSM2497818    0
GSM2497819    0
GSM2497820    0
GSM2497821    0
GSM2497822    0
GSM2497823    0
GSM2497824    0
GSM2497825    0
Name: 1, Length: 108, dt

In [146]:
# Read dataframes
matches_df=pd.read_table('s2-signatures.dir/melanoma-idmatches.txt', index_col='sample')
cpm_table=pd.read_table('s1-expression_data.dir/melanoma-cpm.txt', index_col='gene')

# Initialize results
results = []

# Looooop
for std_i, match in matches_df.items():
    cd_res = geode_jupies.chdir(cpm_table.values, match.values, cpm_table.index, 
                    gamma=.5, sort=False, calculate_sig=False)
    cd_dataframe = pd.DataFrame(cd_res).rename(columns={1: 'gene', 0: 'CD'})
    cd_dataframe['signature'] = std_i
    results.append(cd_dataframe)

# Concatenate and merge
result_dataframe = pd.concat(results)
result_table = result_dataframe.pivot(index='gene', columns='signature', values='CD')

# Write
result_table.to_csv(outfile, sep='\t')

In [ ]:
# create an empty dataframe
cd_results = pd.DataFrame(index=cpm_table.index)
d_std_i_cd = {} # to top up/down genes

for std_i, match in matches.items():
    cd_res = geode.chdir(cpm_table.values, match, cpm_table.index, 
                    gamma=.5, sort=False, calculate_sig=False)
    cd_coefs = np.array(map(lambda x: x[0], cd_res))
    cd_results[std_i] = cd_coefs

    # sort CD in by absolute values in descending order
    srt_idx = np.abs(cd_coefs).argsort()[::-1]
    cd_coefs = cd_coefs[srt_idx][:600]
    sorted_DEGs = cpm_table.index[srt_idx][:600]

    # split up and down
    up_genes = dict(zip(sorted_DEGs[cd_coefs > 0], cd_coefs[cd_coefs > 0]))
    dn_genes = dict(zip(sorted_DEGs[cd_coefs < 0], cd_coefs[cd_coefs < 0]))
    d_std_i_cd[std_i+'-up'] = up_genes
    d_std_i_cd[std_i+'-dn'] = dn_genes
